In [ ]:
from dask.diagnostics import ProgressBar
from dask.distributed import Client
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer

import dask.dataframe as dd
import html
import psutil
import pandas as pd
import pickle as pkl
import re
import time

### Clustering Approach
- K-Means

**Choosing the Number of Clusters**
- Elbow Method: Plot the sum of squared distances of samples to their closest cluster center. The "elbow" point where the rate of decrease sharply changes can suggest a good number of clusters.
- Silhouette Score: Measures how similar an object is to its own cluster compared to other clusters. The higher the silhouette score, the better the clustering.

### 1. Activity-Based Clustering
Cluster by Total Activity: 
- Group users based on their total contributions, including 
    - question count, 
    - answer count, and 
    - comment count.

### 2. Quality and Reputation-Based Clustering
Cluster by Reputation and Scores: 
Group users based on 
- their reputation, 
- average question score, 
- average answer score, and 
- average comment score. 

This can help identify influential contributors.

- Cluster by Accepted Answer Count and Fraction

### 3. Sentiment-Based Clustering
- Cluster by Sentiment Scores: 
Use average sentiment scores from 
- questions, 
- answers, and 
- comments 

to identify users who generally express positive, neutral, or negative sentiments.

### 4. Engagement-Based Clustering
Cluster by 
- Views, 
- UpVotes, and 
- DownVotes: 

Group users based on how their content is received by the community.